## 一些准备工作

In [6]:
# torch packets
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import numpy as np 
from sklearn import  metrics

# torch vision 
from torchvision import datasets, transforms

mnist_path = 'DATA'
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (1.0,))])


train_set = datasets.MNIST(root=mnist_path, train=True, transform=trans, download=True)
test_set = datasets.MNIST(root=mnist_path, train=False, transform=trans)
batch_size = 128
train_loader = torch.utils.data.DataLoader(
                 dataset=train_set,
                 batch_size=batch_size,
                 shuffle=True)
test_loader = torch.utils.data.DataLoader(
                dataset=test_set,
                batch_size=batch_size,
                shuffle=False)

# 模型结构
class mnist_net(nn.Module):
    def __init__(self):
        super(mnist_net, self).__init__() # 为啥？
        self.fc1 = nn.Linear(28* 28 ,  500)
        self.fc2 = nn.Linear(500, 32)
        self.fc3 = nn.Linear(32, 10)
        pass
    
    
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu( self.fc1(x)  )
        x = F.relu( self.fc2(x)  )
        out = self.fc3(x)
        return out 
        pass 
    pass

the_model = mnist_net()
the_model.load_state_dict(torch.load('mnist_net'))

In [8]:
y_true = test_set.targets
y_pred2 = torch.empty(0, dtype= torch.long)
for batch_idx, (x, target) in enumerate(test_loader):
    y_pred_tmp = the_model.forward(x) 
    label_pred = torch.argmax( y_pred_tmp, dim= 1 )
    #print(label_pred)
    #print(target)
    y_pred2 = torch.cat( (y_pred2, label_pred)  ) # 连接两个向量
    
    pass

print('\nAccuracy Score of the Loaded Model: %g' %( metrics.accuracy_score(y_true, y_pred2)))


Accuracy Score of the Loaded Model: 0.9634
